## Visual Transformer for Skin Cancer Detection

In [1]:
import evaluate
import torch
from transformers import ViTForImageClassification, Trainer, TrainingArguments
from transformers import ViTImageProcessor
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import datasets
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import h5py
import cv2
from PIL import Image
from transformers import pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import safetensors.torch
from sklearn.metrics import accuracy_score

In [2]:
MODELS_SAVE_PATH = '../models/'
BASE_PATH = "../data/isic-2024-challenge"

In [3]:

# Train + Valid
df = pd.read_csv(f'{BASE_PATH}/train-metadata.csv')
df = df.ffill()
display(df.head(2))

# Testing
testing_df = pd.read_csv(f'{BASE_PATH}/test-metadata.csv')
testing_df = testing_df.ffill()
display(testing_df.head(2))

/scratch/1392489.1.bil-koo-gpu-pub/ipykernel_2714980/463271929.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{BASE_PATH}/train-metadata.csv')


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455


,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,20.007270,...,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015729,IP_1664139,35.0,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,9.657964,...,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY


In [4]:
#Handle Class Imbalance
print("Class Distribution Before Sampling (%):")
display(df.target.value_counts(normalize=True)*100)
seed = 1
neg_sample = .01
pos_sample = 5.0
# Sampling
positive_df = df.query("target==0").sample(frac=neg_sample, random_state=seed)
negative_df = df.query("target==1").sample(frac=pos_sample, replace=True, random_state=seed)
df = pd.concat([positive_df, negative_df], axis=0).sample(frac=1.0)

print("\nClass Distribution After Sampling (%):")
display(df.target.value_counts(normalize=True)*100)

# Assume df is your DataFrame and 'target' is the column with class labels
class_weights = compute_class_weight('balanced', classes=np.unique(df['target']), y=df['target'])
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Distribution Before Sampling (%):


target
0    99.902009
1     0.097991
Name: proportion, dtype: float64


Class Distribution After Sampling (%):


target
0    67.09645
1    32.90355
Name: proportion, dtype: float64

Class Weights: {0: 0.7451959071624656, 1: 1.519592875318066}


In [5]:
from sklearn.model_selection import StratifiedGroupKFold

training_validation_hdf5 = h5py.File(f"{BASE_PATH}/train-image.hdf5", 'r')

# Reset index to ensure a continuous index
df = df.reset_index(drop=True)
df["fold"] = -1

# Set up the StratifiedGroupKFold with 5 splits
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=seed)

# Assign fold numbers to each data point
for i, (training_idx, validation_idx) in enumerate(sgkf.split(df, y=df.target, groups=df.patient_id)):
    df.loc[validation_idx, "fold"] = int(i)

# Define the train, validation, and test sets
# Use fold 0 for test, fold 1 for validation, and remaining folds for training
training_df = df.query("fold > 1")  # Folds 2, 3, 4 for training
validation_df = df.query("fold == 1")  # Fold 1 for validation
test_df = df.query("fold == 0")  # Fold 0 for testing

# Print the number of samples in each set
print(f"# Num Train: {len(training_df)} | Num Valid: {len(validation_df)} | Num Test: {len(test_df)}")


# Num Train: 3647 | Num Valid: 1066 | Num Test: 1259


In [15]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

transform = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=processor.image_mean, std=processor.image_std),
    ])

def generate_rotations(image):
    rotations = []
    for angle in [0, 90, 180, 270]:
        rotated_image = image.rotate(angle, expand=True) 
        rotations.append(rotated_image)
    return rotations


def preprocess_images(example):
        byte_string = training_validation_hdf5[example["isic_id"]][()]
        nparr = np.frombuffer(byte_string, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)[...,::-1]
        image = Image.fromarray(image)
        rotated_images = generate_rotations(image)
        processed_images = [transform(rotated_image) for rotated_image in rotated_images]
        example['pixel_values'] = processed_images
        example['target'] = [example['target']] * len(processed_images)  # Repeat the label for each rotation
        return example

ds_training = datasets.Dataset.from_pandas(pd.DataFrame(data=training_df))
ds_valid = datasets.Dataset.from_pandas(pd.DataFrame(data=validation_df))
ds_test = datasets.Dataset.from_pandas(pd.DataFrame(data=test_df))


ds_training = ds_training.map(preprocess_images, batched=False)
ds_valid = ds_valid.map(preprocess_images, batched=False)
ds_test = ds_test.map(preprocess_images, batched=False)

Map:   0%|          | 0/3647 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1259 [00:00<?, ? examples/s]

In [17]:
#accuracy metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
     

In [18]:
model_name='google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)

# Make sure its the right tensor types
def collate_fn(batch):
    pixel_values = torch.stack([torch.tensor(example['pixel_values']) for example in batch])
    labels = torch.tensor([example['target'] for example in batch])
    return {'pixel_values': pixel_values, 'labels': labels}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./vit-finetuned-agent0',
    metric_for_best_model = "accuracy",
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=20,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    remove_unused_columns=False,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_training,
    eval_dataset=ds_valid,
    data_collator=collate_fn,
    tokenizer=processor,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
#train da model
trainer.train()

# Save da fine-tuned model
trainer.save_model(MODELS_SAVE_PATH + 'ViT_rotation')

ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/share/pkg.8/academic-ml/fall-2024/install/fall-2024-pyt/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/share/pkg.8/academic-ml/fall-2024/install/fall-2024-pyt/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/share/pkg.8/academic-ml/fall-2024/install/fall-2024-pyt/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr4/cs640/rakin374/.local/lib/python3.11/site-packages/transformers/models/vit/modeling_vit.py", line 797, in forward
    outputs = self.vit(
              ^^^^^^^^^
  File "/share/pkg.8/a

In [ ]:
trainer.eval_dataset=ds_test
trainer.evaluate()

In [ ]:
def process_test_set(ds, num_samples):
    inputs = []
    labels = []
    for i in range(num_samples):
        byte_string = training_validation_hdf5[ds[i]["isic_id"]][()]
        nparr = np.frombuffer(byte_string, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)[...,::-1]
        image = Image.fromarray(image)
        inputs.append(image)
        labels.append(ds[i]["target"]) 
    return inputs, labels

inputs, y_true = process_test_set(ds_test, 1000)
print(inputs[:10])

In [ ]:
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k")
state_dict = safetensors.torch.load_file("vit-finetuned-agent0/checkpoint-228/model.safetensors")
model.load_state_dict(state_dict)

image_classifier = pipeline("image-classification", model,image_processor=processor)
predictions = image_classifier(inputs)
#select highest labels
predictions = [max(item, key=lambda x: x['score'])['label'] for item in predictions]
#convert from LABEL_0, LABEL_1 to 0,1
predictions = [1 if item == 'LABEL_1' else 0 for item in predictions]


In [ ]:
accuracy = accuracy_score(y_true, predictions)
accuracy

In [10]:
#plot confusion matrix
confusion_metric = evaluate.load("confusion_matrix")
confusion_matrix = confusion_metric.compute(predictions=predictions, references=y_true)
matrix = confusion_matrix['confusion_matrix']
if 'labels' in confusion_matrix:
    labels = confusion_matrix['labels']
else:
    labels = np.unique(predictions + y_true)
sns.heatmap(matrix, annot=True, fmt="d", xticklabels=labels, yticklabels=labels, cmap="Blues")
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()

NameError: name 'predictions' is not defined